In [1]:
import time
from datetime import timedelta

import sys
import os
import json
import psycopg2
from dotenv import load_dotenv 
sys.path.append('/home/jovyan/work')
import src.db_table_spotify_history as tbh
# Carregar variáveis de ambiente do arquivo .env
load_dotenv(os.path.join('config', '.env'))
# Configuração da conexão com o banco de dados
user_name = 'User_1'
user = True if user_name == 'User_1'  else False

if user:
    db_config = {
        'host': os.getenv('DB_HOST'),
        'port': os.getenv('DB_PORT'),
        'dbname': os.getenv('DB_NAME'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASSWORD')
    }
    # Diretório onde estão localizados os arquivos JSON
    json_dir = '/home/jovyan/work/data/Spotify_history/'
else:
    db_config = {
        'host': os.getenv('DB_HOST'),
        'port': os.getenv('DB_PORT'),
        'dbname': os.getenv('DB_NAME_2'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASSWORD')
        }
    # Diretório onde estão localizados os arquivos JSON
    json_dir = '/home/jovyan/work/data/jana_spotify_data'

def connect_to_db() -> psycopg2.extensions.connection:
    """
    Função para conectar ao banco de dados PostgreSQL.

    Retorna:
        conn (psycopg2.extensions.connection): Objeto de conexão com o banco de dados.
    """
    try:
        conn = psycopg2.connect(**db_config)
        print("Conexão com o banco de dados estabelecida com sucesso!")
        return conn
    except psycopg2.Error as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

In [2]:
def process_json_files() -> None:
    """
    Processa arquivos JSON e insere os dados no banco de dados PostgreSQL.

    Retorna:
        None
    """
    conn = psycopg2.connect(**db_config)
    
    try:
        for filename in os.listdir(json_dir):
            if filename.endswith('.json'):
                print(f'Inserindo dados do arquivo:{filename}')
                with open(os.path.join(json_dir, filename), 'r') as file:
                    json_data = json.load(file)
                    
                    # Assumindo que json_data é uma lista de dicionários
                    for dicionario in json_data:
                        tbh.insert_data(conn = conn, data = dicionario, your_table_name = 'all_tracks_registry')
                print(f'Dados de {filename} inseridos')
                print('=====')
    finally:
        conn.close()


In [3]:
start_time = time.time()

In [4]:
tbh.create_table_all_tracks_registry(conn=connect_to_db())

Conexão com o banco de dados estabelecida com sucesso!
Tabela criada com sucesso!


In [5]:
process_json_files()

Inserindo dados do arquivo:Streaming_History_Audio_2021-2023_0.json
Dados de Streaming_History_Audio_2021-2023_0.json inseridos
=====
Inserindo dados do arquivo:Streaming_History_Audio_2023-2024_1.json
Dados de Streaming_History_Audio_2023-2024_1.json inseridos
=====
Inserindo dados do arquivo:Streaming_History_Video_2023-2024.json
Dados de Streaming_History_Video_2023-2024.json inseridos
=====


In [6]:
end_time = time.time()
elapsed_time = end_time - start_time
# Converter o tempo decorrido para o formato HH:MM:SS 
print(f"Tempo de processamento: {str(timedelta(seconds=int(elapsed_time)))}")

Tempo de processamento: 0:02:43
